In [ ]:
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

In [ ]:
def SendMail(ImgFileName):
    with open(ImgFileName, 'rb') as f:
        img_data = f.read()

    msg = MIMEMultipart()
    msg['Subject'] = 'Face Emotion_speech_Emo'
    msg['From'] = 'fseapna01@gmal.com'
    msg['To'] = 'fseapna01@gmail.com'

    text = MIMEText("Result")
    msg.attach(text)
    image = MIMEImage(img_data, name=os.path.basename(ImgFileName))
    msg.attach(image)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login('fseapna01@gmail.com', 'bippnarpkmkzbpak')
    s.sendmail('fseapna01@gmail.com', 'fseapna01@gmail.com', msg.as_string())
    s.quit()

In [ ]:
from tkinter import *
from tkinter import filedialog
from PIL import Image
from PIL import ImageTk
import cv2
import imutils
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
global count
count=0
faceClassif = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
with open('model.json', "r") as json_file:
    loaded_model_json = json_file.read()
    emotion_model = model_from_json(loaded_model_json)

emotion_model.load_weights('model_weights.h5')
emotion_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}

def cam():
    global cap
    if selected.get() == 1:
        path_video = filedialog.askopenfilename(filetypes = [
            ("all video format", ".mp4"),
            ("all video format", ".avi")])
        if len(path_video) > 0:
            btnEnd.configure(state="active")
            rad1.configure(state="disabled")
            rad2.configure(state="disabled")

            pathInputVideo = "..." + path_video[-20:]
            lblInfoVideoPath.configure(text=pathInputVideo)
            cap = cv2.VideoCapture(path_video)
            visualizar()
    if selected.get() == 2:
        btnEnd.configure(state="active")
        rad1.configure(state="disabled")
        rad2.configure(state="disabled")
        lblInfoVideoPath.configure(text="")
        cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        visualizar()
def visualizar():
    global cap
    ret, frame = cap.read()
    if ret == True:
        frame = imutils.resize(frame, width=640)
        frame = detection(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(frame)
        img = ImageTk.PhotoImage(image=im)

        lblVideo.configure(image=img)
        lblVideo.image = img
        lblVideo.after(10, visualizar)
    else:
        lblVideo.image = ""
        lblInfoVideoPath.configure(text="")
        rad1.configure(state="active")
        rad2.configure(state="active")
        selected.set(0)
        btnEnd.configure(state="disabled")
        cap.release()

def detection(frame):
    global count
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceClassif.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_gray_frame = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        if maxindex==1 or maxindex==5:
            count=count+1
        else:
            count=count-1
        if count<0:
            count==0
        if count==3:
            cv2.imwrite('img.jpg',frame)   
            SendMail('img.jpg')
            count=0
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return frame


def exit():
    lblVideo.image = ""
    lblInfoVideoPath.configure(text="")
    rad1.configure(state="active")
    rad2.configure(state="active")
    selected.set(0)
    cap.release()
    

    
cap = None
root = Tk()

lblInfo1 = Label(root, text="Face Emotion_speech Recognition", font="bold")
lblInfo1.grid(column=0, row=0, columnspan=2)
selected = IntVar()
rad1 = Radiobutton(root, text="Input video", width=20, value=1, variable=selected, command=cam)
rad2 = Radiobutton(root, text="Webcam", width=20, value=2, variable=selected, command=cam)
rad1.grid(column=0, row=1)
rad2.grid(column=1, row=1)
lblInfoVideoPath = Label(root, text="", width=20)
lblInfoVideoPath.grid(column=0, row=2)

lblVideo = Label(root)
lblVideo.grid(column=0, row=3, columnspan=2)

btnEnd = Button(root, text="Exit", state="disabled", command=exit)
btnEnd.grid(column=0, row=4, columnspan=2, pady=10)

root.mainloop()

1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 30ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\niyas\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\niyas\anaconda3\lib\tkinter\__init__.py", line 839, in callit
    func(*args)
  File "C:\Users\niyas\AppData\Local\Temp\ipykernel_11540\956930415.py", line 53, in visualizar
    frame = detection(frame)
  File "C:\Users\niyas\AppData\Local\Temp\ipykernel_11540\956930415.py", line 88, in detection
    SendMail('img.jpg')
NameError: name 'SendMail' is not defined
